# CAR-Recognition

In [1]:
import cv2                 # deal with images
import numpy as np         
import os                  # deal with direc|
from random import shuffle 
from tqdm import tqdm      # percentage bar


# Train Car

In [ ]:
TRAIN_DIR = 'train_car/'
TEST_DIR = 'test/'
IMG_SIZE = 32
LR = 1e-4

MODEL_NAME = 'cars-{}-{}.model'.format(LR, '6conv')

In [ ]:
def label_img(img):
    word_label = img.split('.')[-3]
    
    if word_label == 'audia4': return [1,0,0,0,0,0,0,0,0,0]
   
    elif word_label == 'fordMustang2017': return [0,1,0,0,0,0,0,0,0,0]
    
    elif word_label == 'benzslk': return [0,0,1,0,0,0,0,0,0,0]
    
    elif word_label == 'bmw325i': return [0,0,0,1,0,0,0,0,0,0]
    
    elif word_label == 'toyotaCamry': return [0,0,0,0,1,0,0,0,0,0]
    
    elif word_label == 'chevroletCamaro2017': return [0,0,0,0,0,1,0,0,0,0]
    
    elif word_label == 'hyundaiSonata2018': return [0,0,0,0,0,0,1,0,0,0]
    
    elif word_label == 'lamboHuracan2017': return [0,0,0,0,0,0,0,1,0,0]
    
    elif word_label == 'volkswagenBeetle2017': return [0,0,0,0,0,0,0,0,1,0]
    
    elif word_label == 'volkswagenGolf2017': return [0,0,0,0,0,0,0,0,0,1]

# Train car 5

In [2]:
TRAIN_DIR = 'train_car_5/'
TEST_DIR = 'test/'
IMG_SIZE = 32
LR = 1e-4

MODEL_NAME = 'cars-{}-{}.model'.format(LR, '6conv')

In [3]:
def label_img(img):
    word_label = img.split('.')[-3]
    
    if word_label == 'volkswagenGolf2017': return [1,0,0,0,0]
   
    elif word_label == 'volkswagenBeetle2017': return [0,1,0,0,0]
    
    elif word_label == 'lamboHuracan2017': return [0,0,1,0,0]
    
    elif word_label == 'hyundaiSonata2018': return [0,0,0,1,0]
    
    elif word_label == 'fordMustang2017': return [0,0,0,0,1]
    


In [4]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
#         img = cv2.imread(path)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data_car.npy', training_data)
    return training_data

In [5]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
#         img = cv2.imread(path)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('test_data_car.npy', testing_data)
    return testing_data

In [6]:
#train_data = create_train_data()
train_data = np.load('train_data_car.npy')

In [ ]:
b = np.load('train_data_car.npy')
print(b)

# 2convnet

In [ ]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

import tensorflow as tf
tf.reset_default_graph()

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 10, activation='softmax')

# convnet = fully_connected(convnet, 1024, activation='relu')
# convnet = dropout(convnet, 0.8)
# #How many class should be push
# convnet = fully_connected(convnet, 10, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')


# 6Convnet

In [7]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

import tensorflow as tf
tf.reset_default_graph()

# Make sure the data is normalized
# img_prep = ImagePreprocessing()
# img_prep.add_featurewise_zero_center()
# img_prep.add_featurewise_stdnorm()

# Create extra synthetic training data by flipping, rotating and blurring the
# images on our data set.
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)
#img_aug.add_random_blur(sigma_max=3.)


#data_augmentation=img_aug,
#data_preprocessing=img_prep,
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1],data_augmentation=img_aug,
                      name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 128, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 256, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 256, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 256, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.5)

convnet = fully_connected(convnet, 5, activation='softmax')




# convnet = fully_connected(convnet, 1024, activation='relu')
# convnet = dropout(convnet, 0.8)
# #How many class should be push
# convnet = fully_connected(convnet, 10, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')


/Library/anaconda3/envs/tflearn/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


# ALEXNET

In [ ]:
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

import tensorflow as tf
tf.reset_default_graph()

# Create extra synthetic training data by flipping, rotating and blurring the
# images on our data set.
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)
img_aug.add_random_blur(sigma_max=3.)


network = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1],
                      name='input')
network = conv_2d(network, 96, 11, strides=4, activation='relu')
network = max_pool_2d(network, 3, strides=2)
network = local_response_normalization(network)
network = conv_2d(network, 256, 5, activation='relu')
network = max_pool_2d(network, 3, strides=2)
network = local_response_normalization(network)
network = conv_2d(network, 384, 3, activation='relu')
network = conv_2d(network, 384, 3, activation='relu')
network = conv_2d(network, 256, 3, activation='relu')
network = max_pool_2d(network, 3, strides=2)
network = local_response_normalization(network)
network = fully_connected(network, 4096, activation='tanh')
network = dropout(network, 0.5)
network = fully_connected(network, 4096, activation='tanh')
network = dropout(network, 0.5)
network = fully_connected(network, 10, activation='softmax')
network = regression(network,  optimizer='momentum', learning_rate=LR, loss='categorical_crossentropy', name='targets')

# Training
model = tflearn.DNN(network, tensorboard_dir='log')

# 6 conv improve

In [ ]:
import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, flatten, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d, avg_pool_2d
from tflearn.layers.estimator import regression

import tensorflow as tf
tf.reset_default_graph()
# Data loading and preprocessing
# from tflearn.datasets import cifar10
# (X, Y), (X_test, Y_test) = cifar10.load_data()
# X, Y = shuffle(X, Y)
# Y = to_categorical(Y)
# Y_test = to_categorical(Y_test)

# Building 'Network In Network'
network = input_data(shape=[None, 32, 32, 1], name='input')
network = conv_2d(network, 192, 5, activation='relu')
network = conv_2d(network, 160, 1, activation='relu')
network = conv_2d(network, 96, 1, activation='relu')
network = max_pool_2d(network, 3, strides=2)
network = dropout(network, 0.5)
network = conv_2d(network, 192, 5, activation='relu')
network = conv_2d(network, 192, 1, activation='relu')
network = conv_2d(network, 192, 1, activation='relu')
network = avg_pool_2d(network, 3, strides=2)
network = dropout(network, 0.5)
network = conv_2d(network, 192, 3, activation='relu')
network = conv_2d(network, 192, 1, activation='relu')
network = conv_2d(network, 10, 1, activation='relu')
network = avg_pool_2d(network, 8)
network = flatten(network)
network = fully_connected(network, 5, activation='softmax')
network = regression(network, optimizer='adam',
                     loss='softmax_categorical_crossentropy',
                     learning_rate=0.001, name='targets')

# Training
model = tflearn.DNN(network, tensorboard_dir='log')
# model.fit(X, Y, n_epoch=50, shuffle=True, validation_set=(X_test, Y_test),
#           show_metric=True, batch_size=128, run_id='cifar10_net_in_net')

In [8]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

In [9]:
train = train_data[:-200]
test = train_data[-200:]

In [10]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

In [11]:
model.fit({'input': X}, {'targets': Y}, n_epoch=60, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, snapshot_epoch=True, show_metric=True, batch_size=128, run_id=MODEL_NAME)

Training Step: 719  | total loss: 1.24876 | time: 1.577s
| Adam | epoch: 060 | loss: 1.24876 - acc: 0.4911 -- iter: 1408/1421
Training Step: 720  | total loss: 1.23961 | time: 2.746s
| Adam | epoch: 060 | loss: 1.23961 - acc: 0.4967 | val_loss: 1.50100 - val_acc: 0.4000 -- iter: 1421/1421
--


# tensorboard --logdir=

In [ ]:
model.save(MODEL_NAME)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:

test_data = process_test_data()

#test_data = np.load('test_data_car.npy')

start = np.random.randint(0,100)
end = start + 12

In [ ]:
c = np.load('test_data_car.npy')
print(c)

In [ ]:
for num,data in enumerate(test_data[start:end]):
    img_data = data[0]
    data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
    model_out = model.predict([data])[0]
model_out
#np.argmax(model_out)

In [ ]:
fig=plt.figure(dpi=100)

for num,data in enumerate(test_data[start:end]):
    
    
    img_num = data[1]
    img_data = data[0]
    
    y = fig.add_subplot(1,4,num+1)
    orig = img_data
    data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
    #model_out = model.predict([data])[0]
    model_out = model.predict([data])[0]
    predict_index = np.argmax(model_out)
    if predict_index == 0: str_label='volkswagenGolf2017'
    elif predict_index == 1: str_label='volkswagenBeetle2017'
    elif predict_index == 2: str_label='lamboHuracan2017'
    elif predict_index == 3: str_label='hyundaiSonata2018'
    elif predict_index == 4: str_label='fordMustang2017'
    
    
    y.imshow(orig,cmap='gray')
    y.imshow(orig)
    plt.title(str_label)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()